# Linear Algebra Special Operations
- **Special matrix operations**: Matrix sign, Kronecker products, special functions
- **Purpose**: Handle specialized matrix computations and transformations
- **Applications**: Control theory, quantum mechanics, tensor operations, ODEs

Key operations:
- Matrix sign function
- Kronecker and Hadamard products
- Matrix balancing
- Block diagonal operations
- Sylvester and Lyapunov equations

In [1]:
import numpy as np
from scipy import linalg
import scipy.linalg as la

# Set print options
np.set_printoptions(precision=4, suppress=True)

print("Special linear algebra operations module loaded")

Special linear algebra operations module loaded


## Matrix Sign Function

For matrix \( A \) with eigenvalues \( \lambda_i \):

\[ \text{sign}(A) = A(A^2)^{-1/2} \]

Eigenvalues of sign(A) are \( \text{sign}(\lambda_i) = \pm 1 \)

**Function**: `scipy.linalg.signm(A)`

**Properties**:
- \( \text{sign}(A)^2 = I \)
- Used in control theory (Riccati equations)
- Separates stable/unstable eigenspaces

**Applications**: Model reduction, invariant subspace computation

In [2]:
# Matrix with positive and negative eigenvalues
A = np.array([[3, 1],
              [1, 2]])

print("Matrix A:")
print(A)

# Compute eigenvalues
eigenvals = linalg.eigvalsh(A)
print(f"\nEigenvalues: {eigenvals}")

# Compute matrix sign
sign_A = linalg.signm(A)
print("\nMatrix sign(A):")
print(sign_A)

# Property: sign(A)^2 = I
sign_A_squared = sign_A @ sign_A
print("\nsign(A)^2:")
print(sign_A_squared)
print(f"Is identity: {np.allclose(sign_A_squared, np.eye(2))}")

# Matrix with mixed eigenvalues
B = np.array([[1, 2],
              [3, -2]])
print("\n\nMatrix B (mixed eigenvalues):")
print(B)
print(f"Eigenvalues: {linalg.eigvals(B).real}")

sign_B = linalg.signm(B)
print("\nsign(B):")
print(sign_B.real)  # May have small imaginary parts

Matrix A:
[[3 1]
 [1 2]]

Eigenvalues: [1.382 3.618]

Matrix sign(A):
[[ 1. -0.]
 [-0.  1.]]

sign(A)^2:
[[ 1. -0.]
 [-0.  1.]]
Is identity: True


Matrix B (mixed eigenvalues):
[[ 1  2]
 [ 3 -2]]
Eigenvalues: [ 2.3723 -3.3723]

sign(B):
[[ 0.5222  0.6963]
 [ 1.0445 -0.5222]]


## Kronecker Product

Tensor product of two matrices:

\[ A \otimes B = \begin{bmatrix} a_{11}B & a_{12}B & \cdots \\ a_{21}B & a_{22}B & \cdots \\ \vdots & \vdots & \ddots \end{bmatrix} \]

**Function**: `scipy.linalg.kron(A, B)` or `np.kron(A, B)`

**Properties**:
- If A is (m×n) and B is (p×q), result is (mp×nq)
- \( (A \otimes B)(C \otimes D) = (AC) \otimes (BD) \)
- \( \text{rank}(A \otimes B) = \text{rank}(A) \cdot \text{rank}(B) \)

**Applications**: Quantum mechanics, PDEs, multi-dimensional problems

In [3]:
# Simple example
A = np.array([[1, 2],
              [3, 4]])
B = np.array([[0, 5],
              [6, 7]])

print("Matrix A (2×2):")
print(A)
print("\nMatrix B (2×2):")
print(B)

# Kronecker product
kron_AB = np.kron(A, B)
print("\nA ⊗ B (4×4):")
print(kron_AB)
# Structure: [[1*B, 2*B],
#             [3*B, 4*B]]

# Verify structure
print("\nTop-left block (a_11 * B = 1*B):")
print(kron_AB[:2, :2])
print("Expected (1*B):")
print(1 * B)

# Identity property: I ⊗ A = block diagonal of A
I2 = np.eye(2)
I_kron_A = np.kron(I2, A)
print("\n\nI ⊗ A (4×4 block diagonal):")
print(I_kron_A)

Matrix A (2×2):
[[1 2]
 [3 4]]

Matrix B (2×2):
[[0 5]
 [6 7]]

A ⊗ B (4×4):
[[ 0  5  0 10]
 [ 6  7 12 14]
 [ 0 15  0 20]
 [18 21 24 28]]

Top-left block (a_11 * B = 1*B):
[[0 5]
 [6 7]]
Expected (1*B):
[[0 5]
 [6 7]]


I ⊗ A (4×4 block diagonal):
[[1. 2. 0. 0.]
 [3. 4. 0. 0.]
 [0. 0. 1. 2.]
 [0. 0. 3. 4.]]


### Kronecker Sum

Related to Kronecker product:

\[ A \oplus B = A \otimes I_n + I_m \otimes B \]

Where A is m×m and B is n×n

**No direct SciPy function** - compute manually

**Property**: \( e^{A \oplus B} = e^A \otimes e^B \)

In [4]:
# Define Kronecker sum manually
def kronecker_sum(A, B):
    """Compute A ⊕ B = A ⊗ I_n + I_m ⊗ B"""
    m = A.shape[0]
    n = B.shape[0]
    Im = np.eye(m)
    In = np.eye(n)
    return np.kron(A, In) + np.kron(Im, B)

# Example
A = np.array([[1, 0],
              [0, 2]])
B = np.array([[3, 1],
              [1, 4]])

print("Matrix A (2×2):")
print(A)
print("\nMatrix B (2×2):")
print(B)

# Kronecker sum
kron_sum = kronecker_sum(A, B)
print("\nA ⊕ B (4×4):")
print(kron_sum)

# Verify property: eigenvalues of A⊕B are λ_i + μ_j
eig_A = linalg.eigvalsh(A)
eig_B = linalg.eigvalsh(B)
eig_sum = linalg.eigvalsh(kron_sum)

print("\nEigenvalues of A:", eig_A)
print("Eigenvalues of B:", eig_B)
print("Eigenvalues of A⊕B:", np.sort(eig_sum))
print("Expected (all λ_i + μ_j):")
expected = sorted([a + b for a in eig_A for b in eig_B])
print(expected)

Matrix A (2×2):
[[1 0]
 [0 2]]

Matrix B (2×2):
[[3 1]
 [1 4]]

A ⊕ B (4×4):
[[4. 1. 0. 0.]
 [1. 5. 0. 0.]
 [0. 0. 5. 1.]
 [0. 0. 1. 6.]]

Eigenvalues of A: [1. 2.]
Eigenvalues of B: [2.382 4.618]
Eigenvalues of A⊕B: [3.382 4.382 5.618 6.618]
Expected (all λ_i + μ_j):
[np.float64(3.381966011250105), np.float64(4.381966011250105), np.float64(5.618033988749895), np.float64(6.618033988749895)]


## Hadamard (Element-wise) Product

Element-by-element multiplication:

\[ (A \odot B)_{ij} = a_{ij} \cdot b_{ij} \]

**Function**: Standard NumPy multiplication `A * B`

**Requirements**: Matrices must have same shape

**Difference from matrix product**:
- Hadamard: \( A \odot B \) (element-wise)
- Matrix: \( AB \) (row-column dot product)

**Applications**: Masking, neural networks, covariance scaling

In [5]:
A = np.array([[1, 2, 3],
              [4, 5, 6]])
B = np.array([[2, 0, 1],
              [1, 3, 2]])

print("Matrix A:")
print(A)
print("\nMatrix B:")
print(B)

# Hadamard product (element-wise)
hadamard = A * B
print("\nHadamard product A ⊙ B:")
print(hadamard)
# Result: [[1*2, 2*0, 3*1], [4*1, 5*3, 6*2]]

# Compare with matrix product (not defined for 2x3 @ 2x3)
print("\nNote: Matrix product A@B not defined for these shapes")

# Square matrices example
C = np.array([[1, 2], [3, 4]])
D = np.array([[2, 0], [1, 2]])

hadamard_CD = C * D
matrix_CD = C @ D

print("\n\nSquare matrices:")
print("C ⊙ D (Hadamard):")
print(hadamard_CD)
print("\nC @ D (Matrix product):")
print(matrix_CD)
print("\nThese are different operations!")

Matrix A:
[[1 2 3]
 [4 5 6]]

Matrix B:
[[2 0 1]
 [1 3 2]]

Hadamard product A ⊙ B:
[[ 2  0  3]
 [ 4 15 12]]

Note: Matrix product A@B not defined for these shapes


Square matrices:
C ⊙ D (Hadamard):
[[2 0]
 [3 8]]

C @ D (Matrix product):
[[ 4  4]
 [10  8]]

These are different operations!


## Block Diagonal Matrices

Create matrix with blocks along diagonal:

\[ \text{block\_diag}(A, B, C) = \begin{bmatrix} A & 0 & 0 \\ 0 & B & 0 \\ 0 & 0 & C \end{bmatrix} \]

**Function**: `scipy.linalg.block_diag(*arrays)`

**Properties**:
- Eigenvalues = union of block eigenvalues
- Determinant = product of block determinants
- Inverse = block diagonal of inverses

**Applications**: Decoupled systems, parallel processing

In [6]:
# Create block diagonal matrix
A = np.array([[1, 2],
              [3, 4]])
B = np.array([[5]])
C = np.array([[6, 7, 8],
              [9, 10, 11],
              [12, 13, 14]])

print("Block A (2×2):")
print(A)
print("\nBlock B (1×1):")
print(B)
print("\nBlock C (3×3):")
print(C)

# Create block diagonal
block_matrix = linalg.block_diag(A, B, C)
print("\nBlock diagonal matrix (6×6):")
print(block_matrix)

# Property: eigenvalues are union of block eigenvalues
eig_A = linalg.eigvals(A)
eig_B = linalg.eigvals(B)
eig_C = linalg.eigvals(C)
eig_block = linalg.eigvals(block_matrix)

print("\nEigenvalues of A:", eig_A)
print("Eigenvalues of B:", eig_B)
print("Eigenvalues of C:", np.sort(eig_C))
print("Eigenvalues of block matrix:", np.sort(eig_block))

Block A (2×2):
[[1 2]
 [3 4]]

Block B (1×1):
[[5]]

Block C (3×3):
[[ 6  7  8]
 [ 9 10 11]
 [12 13 14]]

Block diagonal matrix (6×6):
[[ 1  2  0  0  0  0]
 [ 3  4  0  0  0  0]
 [ 0  0  5  0  0  0]
 [ 0  0  0  6  7  8]
 [ 0  0  0  9 10 11]
 [ 0  0  0 12 13 14]]

Eigenvalues of A: [-0.3723+0.j  5.3723+0.j]
Eigenvalues of B: [5.+0.j]
Eigenvalues of C: [-0.5885+0.j  0.    +0.j 30.5885+0.j]
Eigenvalues of block matrix: [-0.5885+0.j -0.3723+0.j  0.    +0.j  5.    +0.j  5.3723+0.j 30.5885+0.j]


## Matrix Balancing

Transform matrix to reduce condition number:

\[ B = D^{-1}AD \]

Where D is diagonal scaling matrix

**Function**: `scipy.linalg.matrix_balance(A)`

Returns: `(balanced_matrix, transform)`

**Purpose**:
- Improve numerical stability
- Reduce rounding errors
- Preprocessing for eigenvalue computation

**Property**: Same eigenvalues as original matrix

In [7]:
# Create ill-conditioned matrix
A = np.array([[1000, 1],
              [1, 0.001]])

print("Original matrix A:")
print(A)
print(f"Condition number: {np.linalg.cond(A):.2e}")

# Balance the matrix
B, T = linalg.matrix_balance(A)

print("\nBalanced matrix B:")
print(B)
print(f"Condition number: {np.linalg.cond(B):.2e}")

print("\nTransformation matrix T:")
print(T)

# Verify: B = T^-1 @ A @ T
T_inv = np.linalg.inv(T)
B_verify = T_inv @ A @ T
print("\nVerification T^-1 @ A @ T:")
print(B_verify)
print(f"Match: {np.allclose(B, B_verify)}")

# Eigenvalues are preserved
eig_A = linalg.eigvals(A)
eig_B = linalg.eigvals(B)
print(f"\nEigenvalues preserved: {np.allclose(np.sort(eig_A), np.sort(eig_B))}")

Original matrix A:
[[1000.       1.   ]
 [   1.       0.001]]
Condition number: 5.81e+21

Balanced matrix B:
[[1000.       1.   ]
 [   1.       0.001]]
Condition number: 5.81e+21

Transformation matrix T:
[[1. 0.]
 [0. 1.]]

Verification T^-1 @ A @ T:
[[1000.       1.   ]
 [   1.       0.001]]
Match: True

Eigenvalues preserved: True


## Companion Matrix

For polynomial \( p(x) = a_0 + a_1x + \cdots + a_{n-1}x^{n-1} + x^n \):

\[ C = \begin{bmatrix} 0 & 0 & \cdots & 0 & -a_0 \\ 1 & 0 & \cdots & 0 & -a_1 \\ 0 & 1 & \cdots & 0 & -a_2 \\ \vdots & \vdots & \ddots & \vdots & \vdots \\ 0 & 0 & \cdots & 1 & -a_{n-1} \end{bmatrix} \]

**Function**: `scipy.linalg.companion(a)`

**Property**: Eigenvalues of C are roots of polynomial p(x)

**Applications**: Root finding, polynomial analysis

In [8]:
# Polynomial: x^3 - 6x^2 + 11x - 6 = (x-1)(x-2)(x-3)
# Coefficients: [1, -6, 11, -6] for x^3 + ... + constant
# Reverse order for companion: [-6, 11, -6, 1]

# Polynomial coefficients (highest degree first)
poly_coeffs = [1, -6, 11, -6]  # x^3 - 6x^2 + 11x - 6

print("Polynomial: x³ - 6x² + 11x - 6")
print("Known roots: 1, 2, 3")

# Create companion matrix
C = linalg.companion(poly_coeffs)
print("\nCompanion matrix:")
print(C)

# Eigenvalues = polynomial roots
roots = linalg.eigvals(C)
print("\nRoots from eigenvalues:", np.sort(roots.real))

# Verify with numpy polynomial roots
roots_numpy = np.roots(poly_coeffs)
print("Roots from np.roots:", np.sort(roots_numpy))

# Another example: x^2 - 5x + 6 = (x-2)(x-3)
poly2 = [1, -5, 6]
C2 = linalg.companion(poly2)
roots2 = linalg.eigvals(C2)
print("\n\nPolynomial: x² - 5x + 6")
print("Companion matrix:")
print(C2)
print("Roots:", np.sort(roots2.real))

Polynomial: x³ - 6x² + 11x - 6
Known roots: 1, 2, 3

Companion matrix:
[[  6. -11.   6.]
 [  1.   0.   0.]
 [  0.   1.   0.]]

Roots from eigenvalues: [1. 2. 3.]
Roots from np.roots: [1. 2. 3.]


Polynomial: x² - 5x + 6
Companion matrix:
[[ 5. -6.]
 [ 1.  0.]]
Roots: [2. 3.]


## Sylvester Equation

Solve for X in:

\[ AX + XB = Q \]

Where A, B, Q are known matrices

**Function**: `scipy.linalg.solve_sylvester(A, B, Q)`

**Applications**:
- Control theory (controllability, observability)
- Image processing
- Matrix equation solving

**Special case**: When B = -A^T, related to Lyapunov equation

In [9]:
# Solve AX + XB = Q
A = np.array([[3, 2],
              [1, 4]])
B = np.array([[1, 1],
              [0, 2]])
Q = np.array([[1, 0],
              [0, 1]])

print("Matrix A:")
print(A)
print("\nMatrix B:")
print(B)
print("\nMatrix Q:")
print(Q)

# Solve Sylvester equation
X = linalg.solve_sylvester(A, B, Q)

print("\nSolution X:")
print(X)

# Verify: AX + XB = Q
verification = A @ X + X @ B
print("\nVerification AX + XB:")
print(verification)
print(f"\nMatches Q: {np.allclose(verification, Q)}")

Matrix A:
[[3 2]
 [1 4]]

Matrix B:
[[1 1]
 [0 2]]

Matrix Q:
[[1 0]
 [0 1]]

Solution X:
[[ 0.2778 -0.1349]
 [-0.0556  0.1984]]

Verification AX + XB:
[[ 1.  0.]
 [-0.  1.]]

Matches Q: True


## Lyapunov Equation

Special case of Sylvester equation:

**Continuous-time**: \[ AX + XA^T = Q \]

**Discrete-time**: \[ AXA^T - X = Q \]

**Functions**:
- `scipy.linalg.solve_continuous_lyapunov(A, Q)`
- `scipy.linalg.solve_discrete_lyapunov(A, Q)`

**Applications**:
- System stability analysis
- Covariance computation
- Optimal control

**Stability**: If A stable and Q positive definite, solution X is positive definite

In [10]:
# Continuous-time Lyapunov: AX + XA^T = Q
A = np.array([[-1, 2],
              [0, -3]])
Q = np.eye(2)

print("Continuous-time Lyapunov equation: AX + XA^T = Q")
print("\nMatrix A (stable: negative eigenvalues):")
print(A)
print(f"Eigenvalues: {linalg.eigvals(A).real}")

print("\nMatrix Q:")
print(Q)

# Solve continuous Lyapunov
X_cont = linalg.solve_continuous_lyapunov(A, Q)
print("\nSolution X:")
print(X_cont)

# Verify: AX + XA^T = Q
verification = A @ X_cont + X_cont @ A.T
print("\nVerification AX + XA^T:")
print(verification)
print(f"Matches Q: {np.allclose(verification, Q)}")

# Check positive definite (all eigenvalues > 0)
X_eigenvals = linalg.eigvalsh(X_cont)
print(f"\nX is positive definite: {np.all(X_eigenvals > 0)}")
print(f"X eigenvalues: {X_eigenvals}")

Continuous-time Lyapunov equation: AX + XA^T = Q

Matrix A (stable: negative eigenvalues):
[[-1  2]
 [ 0 -3]]
Eigenvalues: [-1. -3.]

Matrix Q:
[[1. 0.]
 [0. 1.]]

Solution X:
[[-0.6667 -0.0833]
 [-0.0833 -0.1667]]

Verification AX + XA^T:
[[1. 0.]
 [0. 1.]]
Matches Q: True

X is positive definite: False
X eigenvalues: [-0.6802 -0.1531]


In [11]:
# Discrete-time Lyapunov: AXA^T - X = Q
A = np.array([[0.5, 0.2],
              [0.1, 0.6]])
Q = np.eye(2)

print("Discrete-time Lyapunov equation: AXA^T - X = Q")
print("\nMatrix A (stable: eigenvalues < 1):")
print(A)
print(f"Eigenvalues: {linalg.eigvals(A).real}")

# Solve discrete Lyapunov
X_disc = linalg.solve_discrete_lyapunov(A, Q)
print("\nSolution X:")
print(X_disc)

# Verify: AXA^T - X = Q
verification = A @ X_disc @ A.T - X_disc
print("\nVerification AXA^T - X:")
print(verification)
print(f"Matches Q: {np.allclose(verification, Q)}")

Discrete-time Lyapunov equation: AXA^T - X = Q

Matrix A (stable: eigenvalues < 1):
[[0.5 0.2]
 [0.1 0.6]]
Eigenvalues: [0.4 0.7]

Solution X:
[[1.5302 0.4059]
 [0.4059 1.6625]]

Verification AXA^T - X:
[[-1. -0.]
 [ 0. -1.]]
Matches Q: False


## Toeplitz and Circulant Matrices

**Toeplitz**: Constant along diagonals
\[ T = \begin{bmatrix} t_0 & t_1 & t_2 \\ t_{-1} & t_0 & t_1 \\ t_{-2} & t_{-1} & t_0 \end{bmatrix} \]

**Circulant**: Special Toeplitz where each row is cyclic shift
\[ C = \begin{bmatrix} c_0 & c_1 & c_2 \\ c_2 & c_0 & c_1 \\ c_1 & c_2 & c_0 \end{bmatrix} \]

**Functions**:
- `scipy.linalg.toeplitz(c, r=None)`
- `scipy.linalg.circulant(c)`
- `scipy.linalg.solve_toeplitz(c, b)`
- `scipy.linalg.solve_circulant(c, b)`

**Applications**: Time series, signal processing, convolution

In [12]:
# Create Toeplitz matrix from column and row
c = [1, 2, 3, 4]  # First column
r = [1, 5, 6, 7]  # First row

T = linalg.toeplitz(c, r)
print("Toeplitz matrix:")
print(T)
# Constant along diagonals

# Symmetric Toeplitz (c = r)
c_sym = [1, 2, 3, 4]
T_sym = linalg.toeplitz(c_sym)
print("\nSymmetric Toeplitz:")
print(T_sym)

# Solve Toeplitz system efficiently
b = np.array([1, 2, 3, 4])
x = linalg.solve_toeplitz((c_sym, c_sym), b)  # Faster than solve(T_sym, b)
print("\nSolution to Tx = b:")
print(x)

# Verify
print(f"Verification T@x = {T_sym @ x}")
print(f"Original b = {b}")
print(f"Match: {np.allclose(T_sym @ x, b)}")

Toeplitz matrix:
[[1 5 6 7]
 [2 1 5 6]
 [3 2 1 5]
 [4 3 2 1]]

Symmetric Toeplitz:
[[1 2 3 4]
 [2 1 2 3]
 [3 2 1 2]
 [4 3 2 1]]

Solution to Tx = b:
[1. 0. 0. 0.]
Verification T@x = [1. 2. 3. 4.]
Original b = [1 2 3 4]
Match: True


In [13]:
# Create circulant matrix
c = [1, 2, 3, 4]
C = linalg.circulant(c)

print("Circulant matrix:")
print(C)
# Each row is cyclic shift of previous

# Property: diagonalized by DFT matrix
from scipy.fft import fft
eigenvals_circ = linalg.eigvals(C)
fft_c = fft(c)

print("\nEigenvalues of circulant:")
print(eigenvals_circ)
print("\nFFT of first column:")
print(fft_c)
print(f"\nMatch: {np.allclose(np.sort(eigenvals_circ), np.sort(fft_c))}")

# Solve circulant system (O(n log n) via FFT)
b = np.array([5, 6, 7, 8])
x = linalg.solve_circulant(c, b)
print("\nSolution to Cx = b:")
print(x)
print(f"Verification: {np.allclose(C @ x, b)}")

Circulant matrix:
[[1 4 3 2]
 [2 1 4 3]
 [3 2 1 4]
 [4 3 2 1]]

Eigenvalues of circulant:
[10.+0.j -2.+2.j -2.-2.j -2.+0.j]

FFT of first column:
[10.-0.j -2.+2.j -2.-0.j -2.-2.j]

Match: False

Solution to Cx = b:
[1.4 0.4 0.4 0.4]
Verification: True


## Hankel Matrix

Constant along anti-diagonals:

\[ H = \begin{bmatrix} h_0 & h_1 & h_2 & h_3 \\ h_1 & h_2 & h_3 & h_4 \\ h_2 & h_3 & h_4 & h_5 \end{bmatrix} \]

**Function**: `scipy.linalg.hankel(c, r=None)`

- **c**: First column
- **r**: Last row (if None, zeros are appended)

**Applications**: System identification, time series analysis

In [14]:
# Create Hankel matrix
c = [1, 2, 3, 4]
r = [4, 5, 6, 7]

H = linalg.hankel(c, r)
print("Hankel matrix:")
print(H)
# Constant along anti-diagonals

# Default (r=None pads with zeros)
H2 = linalg.hankel(c)
print("\nHankel with zero padding:")
print(H2)

# Relationship to Toeplitz
# Hankel = Toeplitz with reversed columns
print("\nNote: Hankel can be obtained from Toeplitz by")
print("reversing column order (anti-diagonal structure)")

Hankel matrix:
[[1 2 3 4]
 [2 3 4 5]
 [3 4 5 6]
 [4 5 6 7]]

Hankel with zero padding:
[[1 2 3 4]
 [2 3 4 0]
 [3 4 0 0]
 [4 0 0 0]]

Note: Hankel can be obtained from Toeplitz by
reversing column order (anti-diagonal structure)


## Matrix Exponential for ODEs

For linear ODE \( \frac{dx}{dt} = Ax \), solution is:

\[ x(t) = e^{At}x_0 \]

**Function**: `scipy.linalg.expm(A*t)`

**Properties**:
- \( e^0 = I \)
- \( \frac{d}{dt}e^{At} = Ae^{At} \)
- \( e^{A(t+s)} = e^{At}e^{As} \) if A commutes with itself

**Applications**: Continuous-time systems, Markov processes

In [15]:
# Solve dx/dt = Ax with x(0) = x0
A = np.array([[-1, 2],
              [0, -2]])
x0 = np.array([1, 0])

print("System: dx/dt = Ax")
print("Matrix A:")
print(A)
print(f"\nInitial condition x(0): {x0}")

# Compute solution at different times
times = [0, 0.5, 1.0, 2.0]

print("\nSolution x(t) = e^(At) @ x0:")
for t in times:
    expm_At = linalg.expm(A * t)
    x_t = expm_At @ x0
    print(f"  t={t:.1f}: x={x_t}")

# Verify exponential property: e^0 = I
expm_0 = linalg.expm(A * 0)
print("\ne^(A*0):")
print(expm_0)
print(f"Is identity: {np.allclose(expm_0, np.eye(2))}")

System: dx/dt = Ax
Matrix A:
[[-1  2]
 [ 0 -2]]

Initial condition x(0): [1 0]

Solution x(t) = e^(At) @ x0:
  t=0.0: x=[1. 0.]
  t=0.5: x=[0.6065 0.    ]
  t=1.0: x=[0.3679 0.    ]
  t=2.0: x=[0.1353 0.    ]

e^(A*0):
[[1. 0.]
 [0. 1.]]
Is identity: True


## Special Matrix Constructors

SciPy provides constructors for special matrices:

| Function | Description | Size |
|----------|-------------|------|
| `hilbert(n)` | Hilbert matrix (ill-conditioned) | n×n |
| `invhilbert(n)` | Inverse Hilbert | n×n |
| `pascal(n)` | Pascal matrix | n×n |
| `invpascal(n)` | Inverse Pascal | n×n |
| `dft(n)` | Discrete Fourier Transform matrix | n×n |
| `hadamard(n)` | Hadamard matrix | n×n (n power of 2) |
| `leslie(f, s)` | Leslie matrix (population) | n×n |

**Note**: These are in `scipy.linalg` submodule

In [16]:
# Hilbert matrix (very ill-conditioned)
H = linalg.hilbert(4)
print("Hilbert matrix (4×4):")
print(H)
print(f"Condition number: {np.linalg.cond(H):.2e}")

# Inverse Hilbert
H_inv = linalg.invhilbert(4)
print("\nInverse Hilbert:")
print(H_inv)

# Verify
identity = H @ H_inv
print(f"\nH @ H_inv is identity: {np.allclose(identity, np.eye(4))}")

# Pascal matrix
P = linalg.pascal(4)
print("\nPascal matrix (4×4):")
print(P)

# DFT matrix
F = linalg.dft(4)
print("\nDFT matrix (4×4):")
print(np.round(F, 4))  # Complex values

# Hadamard matrix (n must be power of 2)
Had = linalg.hadamard(4)
print("\nHadamard matrix (4×4):")
print(Had)
print(f"\nOrthogonal (scaled): {np.allclose(Had.T @ Had, 4 * np.eye(4))}")

Hilbert matrix (4×4):
[[1.     0.5    0.3333 0.25  ]
 [0.5    0.3333 0.25   0.2   ]
 [0.3333 0.25   0.2    0.1667]
 [0.25   0.2    0.1667 0.1429]]
Condition number: 1.55e+04

Inverse Hilbert:
[[   16.  -120.   240.  -140.]
 [ -120.  1200. -2700.  1680.]
 [  240. -2700.  6480. -4200.]
 [ -140.  1680. -4200.  2800.]]

H @ H_inv is identity: True

Pascal matrix (4×4):
[[ 1  1  1  1]
 [ 1  2  3  4]
 [ 1  3  6 10]
 [ 1  4 10 20]]

DFT matrix (4×4):
[[ 1.+0.j  1.+0.j  1.+0.j  1.+0.j]
 [ 1.+0.j  0.-1.j -1.-0.j -0.+1.j]
 [ 1.+0.j -1.-0.j  1.+0.j -1.-0.j]
 [ 1.+0.j -0.+1.j -1.-0.j  0.-1.j]]

Hadamard matrix (4×4):
[[ 1  1  1  1]
 [ 1 -1  1 -1]
 [ 1  1 -1 -1]
 [ 1 -1 -1  1]]

Orthogonal (scaled): True


## Summary: Key Takeaways

✓ **Matrix sign**: `signm(A)` for control theory; satisfies sign(A)² = I  
✓ **Kronecker product**: `kron(A, B)` for tensor operations; size (mp×nq)  
✓ **Hadamard product**: `A * B` element-wise multiplication  
✓ **Block diagonal**: `block_diag(*arrays)` creates diagonal blocks  
✓ **Matrix balancing**: `matrix_balance(A)` improves numerical stability  
✓ **Companion matrix**: `companion(poly)` converts polynomial to matrix  
✓ **Sylvester equation**: `solve_sylvester(A, B, Q)` solves AX + XB = Q  
✓ **Lyapunov equations**: `solve_continuous_lyapunov()`, `solve_discrete_lyapunov()`  
✓ **Structured matrices**: `toeplitz()`, `circulant()`, `hankel()` with fast solvers  
✓ **Special constructors**: `hilbert()`, `pascal()`, `dft()`, `hadamard()`  

### Matrix Structure Comparison:

| Structure | Pattern | Solver | Complexity |
|-----------|---------|--------|------------|
| Toeplitz | Constant diagonals | `solve_toeplitz()` | O(n²) |
| Circulant | Cyclic rows | `solve_circulant()` | O(n log n) |
| Hankel | Constant anti-diagonals | Standard solve | O(n³) |
| Block diagonal | Diagonal blocks | Per-block solve | O(block³) |

### When to Use:

| Operation | Use Case |
|-----------|----------|
| `signm()` | System stability, eigenspace separation |
| `kron()` | Multi-dimensional PDEs, quantum mechanics |
| `block_diag()` | Decoupled systems, parallel computation |
| Lyapunov | Stability analysis, covariance equations |
| Sylvester | Control systems, matrix equations |
| Circulant | Convolution, periodic signals |
| Companion | Polynomial root finding |

### Best Practices:
1. **Use specialized solvers** for structured matrices (faster)
2. **Balance matrices** before eigenvalue computation
3. **Exploit structure** - circulant is faster than Toeplitz
4. **Check stability** - negative eigenvalues for continuous Lyapunov
5. **Kronecker operations** can explode matrix size - use sparingly

### Next Steps:
- Explore sparse matrix operations for large systems
- Learn iterative solvers for huge matrices
- Study numerical ODE solvers (scipy.integrate)